# Candidate Extraction : Diseases

This notebook is meant for in-house demonstration of candidate extraction and featurization of tables. It assumes an input file in XHTML format, a strict form of HTML that coincides with XML structure, allowing for easy display (HTML) and safe tree traversal (XML).

In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Candidate Extraction

First, import the 'HTMLParser' class to read HTML tables

In [10]:
from snorkel.parser import HTMLParser
html_parser = HTMLParser(path='data/diseases/diseases.xhtml')

/Users/bradenhancock/anaconda/lib/python2.7/site-packages/sqlalchemy/ext/declarative/clsregistry.py:120: SAWarning: This declarative base already contains a class with the same class name and module name as snorkel.models.context.Context, and will be replaced in the string-lookup table.
  item.__name__
[autoreload of snorkel.models.context failed: Traceback (most recent call last):
  File "/Users/bradenhancock/anaconda/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
InvalidRequestError: Table 'context' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.
]


The "TableParser" class divides the html doc into cells, adding a 'cell_id' attribute to each cell for future traversal, and creating "Cell" objects that have attributes such as row number, column number, html tag, html attributes, and any tags/attributes on a cells ancestors in the table.

In [11]:
from snorkel.parser import TableParser
table_parser = TableParser()

As usual, pass these to a Corpus object for digestion.

In [12]:
# from snorkel.parser import Corpus
# %time corpus = Corpus(html_parser, table_parser)

from snorkel.parser import CorpusParser
cp = CorpusParser(html_parser, table_parser)
%time corpus = cp.parse_corpus(name='Diseases Corpus')

CPU times: user 171 ms, sys: 89.5 ms, total: 260 ms
Wall time: 379 ms


In [13]:
doc = corpus.documents[0]
print doc
print doc.tables[0]
print doc.tables[0].cells[0]
print doc.tables[0].cells[0].phrases[0]

Document('0', Corpus (Diseases Corpus))
Table('0', 0)
Cell('0', 0, 0, 'Disease')
Phrase('0', 0, 0, 0, u'Disease')


Load the good 'ole disease dictionary for recognizing disease names.

In [14]:
from load_dictionaries import load_disease_dictionary

# Load the disease phrase dictionary
diseases = load_disease_dictionary()
print "Loaded %s disease phrases!" % len(diseases)

Loaded 507899 disease phrases!


Here we use a new CandidateSpace object, CellNgrams. It inherits from Ngrams, and ensures that the Table context object is broken up into cells before being passed into the usual routine for pulling out Ngrams.

In [15]:
from snorkel.candidates import TableNgrams
from snorkel.matchers import DictionaryMatch

# Define a candidate space
ngrams = TableNgrams(n_max=3)

# Define a matcher
disease_matcher = DictionaryMatch(d=diseases, longest_match_only=False)

Passing the CandidateSpace, Matcher, and Context objects to a Candidates object, extraction is performed, and we see that a number of disease CellNgrams are returned.

In [16]:
# With new Candidates object:
# from snorkel.candidates import Candidates
# %time candidates = Candidates(table_ngrams, disease_matcher, corpus.get_contexts())

# With old Candidates object:
from snorkel.candidates import CandidateExtractor
ce = CandidateExtractor(ngrams, disease_matcher)
%time candidates = ce.extract(corpus.get_tables(), name='all')

for cand in candidates: print cand

CPU times: user 56.3 ms, sys: 3.79 ms, total: 60.1 ms
Wall time: 58.2 ms
Ngram("Disease", context=None, chars=[0,6], words=[0,0])
Ngram("Location", context=None, chars=[0,7], words=[0,0])
Ngram("Polio", context=None, chars=[0,4], words=[0,0])
Ngram("Chicken Pox", context=None, chars=[0,10], words=[0,1])
Ngram("plague", context=None, chars=[31,36], words=[3,3])
Ngram("Scurvy", context=None, chars=[0,5], words=[0,0])
Ngram("Problem", context=None, chars=[0,6], words=[0,0])
Ngram("Arthritis", context=None, chars=[0,8], words=[0,0])
Ngram("Yellow Fever", context=None, chars=[0,11], words=[0,1])
Ngram("Fever", context=None, chars=[7,11], words=[1,1])
Ngram("Hypochondria", context=None, chars=[0,11], words=[0,0])
Ngram("Disease", context=None, chars=[0,6], words=[0,0])
Ngram("Location", context=None, chars=[0,7], words=[0,0])
Ngram("Polio", context=None, chars=[0,4], words=[0,0])
Ngram("Chicken Pox", context=None, chars=[0,10], words=[0,1])
Ngram("plague", context=None, chars=[31,36], words=

### Feature Generation

We can then generate features on our set of candidates, including *new and improved* table features!

In [40]:
from snorkel.features import TableNgramFeaturizer
featurizer = TableNgramFeaturizer()
featurizer.fit_transform(candidates)

<22x113 sparse matrix of type '<type 'numpy.float64'>'
	with 440 stored elements in LInked List format>

In [41]:
featurizer.print_features(candidates[:1])

DDLIB_WORD_SEQ_[Disease]
DDLIB_LEMMA_SEQ_[disease]
DDLIB_POS_SEQ_[NN]
DDLIB_DEP_SEQ_[ROOT]
DDLIB_W_LEFT_1_[disease]
DDLIB_W_LEFT_POS_1_[NN]
DDLIB_STARTS_WITH_CAPTIAL
DDLIB_LENGTH_1
TABLE_ROW_NUM_0
TABLE_COL_NUM_1
TABLE_HTML_TAG_th
TABLE_HTML_ANC_TAG_tr
TABLE_HTML_ANC_TAG_tbody
TABLE_HTML_ANC_TAG_table
TABLE_HTML_ANC_TAG_body
TABLE_HTML_ANC_ATTR_align=left
TABLE_HTML_ANC_ATTR_size=5
TABLE_HTML_ANC_ATTR_font=blue


Ta-da! Next up: feeding these features into the learning machine.